## Cодержание:
* [Подключение модулей](#first)
* [Загрузка и изучение данных](#second)
* [Базовый анализ данных](#third)
* [Предобработка данных](#fourth)
* [Обучение модели](#fifth)
* [Тестирование модели](#sixth)
* [Выводы](#seventh)

# Команда: Бета Банк


**Цель:** Создать CLTV модель, которая будет выдавать вероятности перехода в каждый из 17 продуктовых кластеров в течение 12 месяцев.

Альфа-Банком предоставлены следующие **данные**:

-   Тренировочный датасет `train_data.pqt` содержит данные о 200 000 клиентах банка и их целевых переменных за три последовательных месяца (month_1, month_2, month_3)
-   Тестовый датасет `test_data.pqt` записи о 100 000 клиентах за 3 последовательных месяца (month_4, month_5, month_6)
   
-   Продуктовый кластер, в котором клиент будет находится через год - `end_cluster`. Необходимо получить вероятности перехода клиента в продуктовые кластеры для последнего месяца (month_6).

  
-    Метрикой качества выступает **ROC-AUC**.

Данные о клиентах и маскированы.

## Подключение модулей <a class="anchor" id="first"></a>

In [2]:
# !pip install catboost

In [3]:
# работа с ОС
import os
import warnings
import time
# работа с данными
import json
import numpy as np
import pandas as pd
from typing import Optional, Dict, Tuple
# визуализация
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from matplotlib.ticker import MaxNLocator
# работа с ML
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# модели машинного обучения
# import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
# обработка ошибок
from pyarrow import ArrowInvalid
# настройка среды выполнения
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', 93)

## Загрузка и изучение данных <a class="anchor" id="second"></a>

In [4]:
def read_df(filepath: str) -> Optional[pd.DataFrame]:
    """
    Функция для чтения датасета по указанному пути

    args:
    filepath (str) - путь к файлу .pqt / .parquet

    return:
    датасет в формате pd.DataFrame, если найден файл по указанному пути
    """
    if os.path.exists(filepath):
        try:
            result: pd.DataFrame = pd.read_parquet(filepath, engine='auto')
            print(f"Файл {filepath} успешно открыт")
            return result
        except ArrowInvalid:
            print("Неверный тип файла. Поддерживаемый - .pqt или .parquet")
            return
    print(f"Файл не найден по пути {filepath}")
    return

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
# Путь до файла train_df
path_train_df = "data/train_data.pqt"
# Путь до файла test_df
path_test_df = "data/test_data.pqt"

# чтение файлов
train_df = read_df(path_train_df)
test_df = read_df(path_test_df)

# объединение файлов
df = pd.concat([train_df, test_df], ignore_index=True)

# меняем в test_df нумерацию месяца
df['date'] = df['date'].replace({'month_4': 'month_1', 'month_5': 'month_2', 'month_6': 'month_3'})


display(train_df.head(2))
display(test_df.head(2))
display(df.head(4))

Файл data/train_data.pqt успешно открыт
Файл data/test_data.pqt успешно открыт


,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.7448,0.7055,1.2872,0.7481,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.9512,0.5687,0.8976,0.5536,0.7744,0.9365,0.2960,0.9679,"{α, γ}",{other}
1,0,month_2,1.0496,0.8319,2.4586,1.0538,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.9488,0.4997,0.7850,0.5519,0.6966,0.9902,0.2989,0.9460,"{α, γ}",{other}


,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.0962,0.3355,-0.1260,-0.0956,channel_code_12,city_14,city_type_0,None,...,0.0110,0.9461,0.4078,-0.1540,0.5489,0.5410,0.0317,0.2573,0.5614,{α}
1,200000,month_5,-0.0243,-0.0598,-0.1243,-0.0234,channel_code_12,city_14,city_type_0,None,...,0.0068,0.9453,0.3963,-0.1505,0.5495,0.5521,0.2378,0.2642,0.7152,{α}


,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.7448,0.7055,1.2872,0.7481,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.9512,0.5687,0.8976,0.5536,0.7744,0.9365,0.2960,0.9679,"{α, γ}",{other}
1,0,month_2,1.0496,0.8319,2.4586,1.0538,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.9488,0.4997,0.7850,0.5519,0.6966,0.9902,0.2989,0.9460,"{α, γ}",{other}
2,0,month_3,0.6927,0.7403,0.4300,0.6957,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.9465,0.4422,0.8770,0.5510,0.6632,0.8101,0.2948,0.9570,"{α, γ}",{other}
3,1,month_1,-0.0816,-0.0919,-0.1140,-0.0809,channel_code_2,city_14,city_type_0,None,...,0.9453,0.4078,0.3693,0.5671,0.7855,-0.1840,0.2535,0.4625,{other},{other}


## Базовый анализ данных <a class="anchor" id="third"></a>

In [10]:
print(f"Количество записей в тренировочных данных: {len(train_df)}")
print(f"Количество записей в тестовых данных: {len(test_df)}")

Количество записей в тренировочных данных: 600000
Количество записей в тестовых данных: 290120


В тестовой выборке у 9880 клиентов нет информации по 4 месяцу.

In [11]:
df.dtypes.value_counts()

float64    81
object     11
int64       1
Name: count, dtype: int64

Посмотрим на нечисловые столбцы

In [13]:
list(df.select_dtypes('object'))

['date',
 'channel_code',
 'city',
 'city_type',
 'index_city_code',
 'ogrn_month',
 'ogrn_year',
 'okved',
 'segment',
 'start_cluster',
 'end_cluster']

Из 11 столбцов: 9 - категориальные признаки, столбец с датой (месяц) и столбец для предсказания. Остальные признаки - числовые.

In [12]:
df[df.duplicated(subset=df.columns.difference(['id']), keep=False)].shape

(44530, 93)

В датасете есть дубликаты, но обучение с ними и без них впоследствии не повлияло на результат. Пока оставляем.

In [14]:
df['start_cluster'].value_counts()

start_cluster
{α}          491192
{}           107072
{α, η}        57458
{α, γ}        43129
{other}       42545
{α, β}        13498
{α, δ}        11350
{α, ε}         7464
{α, θ}         6348
{α, ψ}         3876
{α, μ}         1883
{α, ε, η}      1620
{α, ε, θ}      1083
{α, λ}          997
{α, ε, ψ}       462
{λ}             114
{α, π}           29
Name: count, dtype: int64

Распределение основного признака (начального кластера, из которого будет осуществляться переход в конечный кластер, являющийся целевой переменной) имеет сильный дизбаланс, но при этом отражает актуальную картину распределения классов.

In [15]:
df['end_cluster'].value_counts()

end_cluster
{α}          318670
{}           122366
{other}       47197
{α, η}        40810
{α, γ}        34471
{α, β}        11203
{α, θ}         6511
{α, ε}         5450
{α, δ}         4078
{α, ψ}         2929
{α, μ}         2123
{α, ε, η}      1674
{α, ε, θ}      1125
{α, λ}          873
{α, ε, ψ}       361
{λ}             140
{α, π}           19
Name: count, dtype: int64

Распределение целевой переменной также имеет дизбаланс. В нашем решении, мы пробовали обучать датасетах: с дизабалансом, с балансом по мажорному классу, с использованием SMOTE, с балансом при указании конкретного распределения (учитывая вес), с полным балансом. После всех тестов, самый высокий результат мы получили при дизбалансе.

## Предобработка данных <a class="anchor" id="fourth"></a>

### Восстановление числовых признаков

Сначала все пропуски в числовых признаках заполнили средним значением. Пробовали также заполнять средним значением с учетом категориального признака okved. Кроме этого, мы заполняли с помощью KKNImputer, IterativeImputer, написали собственный алгоритм для заполнения признаков каждого клиента по отдельности (по линейной зависимости данных за разные месяца). Результат был выше при первом способе. 

In [16]:
numeric_cols = df.drop(columns=['id']).select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    # получением минимального возможного значения признака
    mean_val = df[col].mean()
    # заполнение пропущенных значений минимальным значением признака
    df[col].fillna(mean_val, inplace=True)

Введение дополнительного признака "среднего" по каждой операции каждого клиента за каждый месяц. Так как признак суммы имеет большое количество одного уникального значения. 

In [17]:
df['avg_a_oper_1m'] = df['sum_a_oper_1m'] / df['cnt_a_oper_1m']
df['avg_b_oper_1m'] = df['sum_b_oper_1m'] / df['cnt_b_oper_1m']
df['avg_c_oper_1m'] = df['sum_c_oper_1m'] / df['cnt_c_oper_1m']

df['avg_deb_d_oper_1m'] = df['sum_deb_d_oper_1m'] / df['cnt_deb_d_oper_1m']
df['avg_cred_d_oper_1m'] = df['sum_cred_d_oper_1m'] / df['cnt_cred_d_oper_1m']

df['avg_deb_e_oper_1m'] = df['sum_deb_e_oper_1m'] / df['cnt_deb_e_oper_1m']
df['avg_cred_e_oper_1m'] = df['sum_cred_e_oper_1m'] / df['cnt_cred_e_oper_1m']


df['avg_deb_f_oper_1m'] = df['sum_deb_f_oper_1m'] / df['cnt_deb_f_oper_1m']
df['avg_cred_f_oper_1m'] = df['sum_cred_f_oper_1m'] / df['cnt_cred_f_oper_1m']

df['avg_deb_g_oper_1m'] = df['sum_deb_g_oper_1m'] / df['cnt_deb_g_oper_1m']
df['avg_cred_g_oper_1m'] = df['sum_cred_g_oper_1m'] / df['cnt_cred_g_oper_1m']

df['avg_deb_h_oper_1m'] = df['sum_deb_h_oper_1m'] / df['cnt_deb_h_oper_1m']
df['avg_cred_h_oper_1m'] = df['sum_cred_h_oper_1m'] / df['cnt_cred_h_oper_1m']


df['avg_a_oper_3m'] = df['sum_a_oper_3m'] / df['cnt_a_oper_3m']
df['avg_b_oper_3m'] = df['sum_b_oper_3m'] / df['cnt_b_oper_3m']
df['avg_c_oper_3m'] = df['sum_c_oper_3m'] / df['cnt_c_oper_3m']

df['avg_deb_d_oper_3m'] = df['sum_deb_d_oper_3m'] / df['cnt_deb_d_oper_3m']
df['avg_cred_d_oper_3m'] = df['sum_cred_d_oper_3m'] / df['cnt_cred_d_oper_3m']

df['avg_deb_e_oper_3m'] = df['sum_deb_e_oper_3m'] / df['cnt_deb_e_oper_3m']
df['avg_cred_e_oper_3m'] = df['sum_cred_e_oper_3m'] / df['cnt_cred_e_oper_3m']

df['avg_deb_f_oper_3m'] = df['sum_deb_f_oper_3m'] / df['cnt_deb_f_oper_3m']
df['avg_cred_f_oper_3m'] = df['sum_cred_f_oper_3m'] / df['cnt_cred_f_oper_3m']

df['avg_deb_g_oper_3m'] = df['sum_deb_g_oper_3m'] / df['cnt_deb_g_oper_3m']
df['avg_cred_g_oper_3m'] = df['sum_cred_g_oper_3m'] / df['cnt_cred_g_oper_3m']

df['avg_deb_h_oper_3m'] = df['sum_deb_h_oper_3m'] / df['cnt_deb_h_oper_3m']
df['avg_cred_h_oper_3m'] = df['sum_cred_h_oper_3m'] / df['cnt_cred_h_oper_3m']

In [18]:
def restore_cal(x: pd.Series) -> pd.Series:
    """
    Функция восстановления категориального признака последним значением для
    конкретного клиента

    args:
    x (pd.Series) - значения рассматриваемого признака в группе

    return:
    Возвращается группа с восстановленным пропущенным значением
    """
    # если есть пропущеные значения
    if x.isna().any() and not x.isna().all():
        # заполняем пропущенное значение последним в группе
        return x.fillna(x.dropna().iloc[-1])
    # если все пропуски, заполняем 'missing'
    elif x.isna().all():
        return x.fillna('missing')
    # если нет пропусков, возвращаем исходную группу
    return x

Восстановление категориальных признаков

In [19]:
cat_columns_to_restore = ['channel_code', 'city', 'city_type', 'ogrn_month', 'ogrn_year', 'okved', 'segment']
for column in cat_columns_to_restore:
    start_time = time.time()
    df[column] = df.groupby('id')[column].apply(restore_cal).reset_index()[column]
    end_time = time.time()
    print(f"Колонка - {column} - восстановлена за {end_time - start_time}")

Колонка - channel_code - восстановлена за 67.60859107971191
Колонка - city - восстановлена за 69.85233616828918
Колонка - city_type - восстановлена за 69.79127621650696
Колонка - ogrn_month - восстановлена за 64.58769083023071
Колонка - ogrn_year - восстановлена за 65.38992476463318
Колонка - okved - восстановлена за 65.90970039367676
Колонка - segment - восстановлена за 63.817808866500854


У клиента категориальные признаки повторяются в разных месяцах (например, city_type (тип города)), кроме start_cluster и end_cluster.
Отсюда следует, что пропущенные категориальные признаки можно восстановить последним значением признака для клиента. Если у клиента нет совсем никакой информации по признаку, то заполняем его новым значением - missing. 

При проверки теории о совпадении категориальных признаков за разные месяца мы получили следующие результаты:
Процент совпадения категориального признака в разные месяца:

1. channel_code - 99.98%
2. city - 99.75%
3. city_type - 99.32%
4. index_city_code - 99.59%
5. ogrn_month - 99.99%
6. ogrn_year - 99.99%
7. okved - 99.62%
8. segment - 99.8%
9. start_cluster - 80.04%
10. end_cluster - 88.41%

Еще из интересной статистики - шанс, что клиент остаётся в том же кластере с 2 по 3 месяц: 93.3%

### Создание таблицы с 3 месяцами

Объединяем три строки данных одного клиента в одну строку, чтобы модель видела связь между данными

In [20]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "ogrn_month", "ogrn_year",
]
cat_cols_month_1 = [f'{col}_month_1' for col in cat_cols]
cat_cols_month_2 = [f'{col}_month_2' for col in cat_cols]
# сводная таблица по клиентам и месяцам
df = df.pivot_table(index='id', columns='date', aggfunc='first')
df.columns = [f'{col[0]}_{col[1]}' for col in df.columns]
df.reset_index(inplace=True)
df = df.drop(
    columns=['end_cluster_month_1', 'end_cluster_month_2'] + cat_cols_month_1 + cat_cols_month_2,
    axis=0)
categorical_columns = df.select_dtypes(include=['object']).columns

# заполняем пропущенные элементы новой категорией, так как у клиентов в тесте может не быть 4 месяца
df[categorical_columns] = df[categorical_columns].fillna("missing")

Самое главное - проследить за изменениями признаков за эти 3 месяца. Создаем таблицу со всеми признаками за каждый месяц.

In [21]:
df.head()

,id,avg_a_oper_1m_month_1,avg_a_oper_1m_month_2,avg_a_oper_1m_month_3,avg_a_oper_3m_month_1,avg_a_oper_3m_month_2,avg_a_oper_3m_month_3,avg_b_oper_1m_month_1,avg_b_oper_1m_month_2,avg_b_oper_1m_month_3,...,sum_deb_h_oper_3m_month_3,sum_of_paym_1y_month_1,sum_of_paym_1y_month_2,sum_of_paym_1y_month_3,sum_of_paym_2m_month_1,sum_of_paym_2m_month_2,sum_of_paym_2m_month_3,sum_of_paym_6m_month_1,sum_of_paym_6m_month_2,sum_of_paym_6m_month_3
0,0,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,0.8770,0.5115,0.4864,0.4805,0.9423,0.6457,0.4036,0.5360,0.5364,0.6132
1,1,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,0.0432,0.0520,0.0336,0.0395,0.0141,-0.0576,-0.0921,0.0438,0.0350,0.0252
2,2,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,-0.1656,-0.2919,-0.2907,-0.2883,-0.2558,-0.2679,-0.2559,-0.2871,-0.2850,-0.2807
3,3,-0.4493,-0.4528,-0.4493,-0.9934,-0.9934,-0.9934,-0.0690,-0.0693,-0.0690,...,-0.1656,-0.2428,-0.2629,-0.2733,-0.2740,-0.2740,-0.2740,-0.2688,-0.2944,-0.2944
4,4,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,-0.0783,-0.1246,-0.1219,-0.1289,-0.1038,-0.1342,-0.1667,-0.1300,-0.1340,-0.1428


In [22]:
# получение только числовых признаков
numeric_cols = df.drop(columns=['id']).select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    # получением минимального возможного значения признака
    mean_val = df[col].mean()
    # заполнение пропущенных значений минимальным значением признака
    df[col].fillna(mean_val, inplace=True)

Заполняем средним. Появились пропущенные значения, так как у клиентов может не быть записи за 4 месяц.

### Восстановление start_claster с помощью CatBoost 

При тестировании мы пробовали CatBoost, LightGBM, XGBoost, мультислойный перцептрон на pytorch, ансамбль из моделей (CatBoost, LightGBM, XGBoost). Лучший результат показал CatBoost при меньшем времени обучения.

In [23]:
# выделяем тренировочные данные (без пропущенных значений целевой переменной)
train_data = df[df['start_cluster_month_3'] != 'missing'].drop(
    ['id', 'end_cluster_month_3'], axis=1)
# выделяем тестовые данные (пропущенные значения целевой переменной)
predict_data = df[df['start_cluster_month_3'] == 'missing'].drop(
    ['id', 'end_cluster_month_3'], axis=1)

# получаем признаки для обучения и целевую переменную
X = train_data.drop('start_cluster_month_3', axis=1)
y = train_data['start_cluster_month_3']

# разбиваем на подвыборки обучения и тестирования 80/20
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

#### Обучение CatBoost

In [24]:
# архитектура модели
catboost_model_start_cluster = CatBoostClassifier(
    iterations=1024,            # кол-во итераций обучения
    depth=6,                    # рекомендованная глубина модели
    learning_rate=0.075,        # скорость обучения
    random_seed=47,             # сид для воспроизводимости результата
    loss_function='MultiClass', # тип модели или функция ошибки
    task_type="GPU",            # обучение на видеокарте
    devices='0',
    early_stopping_rounds=20    # регуляризация ранней остановкой в случае
                                # отстутсвия изменения функции ошибки 20
                                # итераций подряд
    )

In [25]:
def train_catboost(model: CatBoostClassifier,
                   x_train: pd.DataFrame, y_train: pd.Series,
                   x_val: pd.DataFrame, y_val: pd.Series,
                   cat_names: pd.core.indexes.base.Index,
                   model_name: str,
                   verbose_step: int = 100) -> pd.DataFrame:
    """
    Функция обучения, тестирования и сохранения модели catboost

    args:
    model (CatBoostClassifier) - модель catboost
    x_train (pd.DataFrame) - датафрейм для обучения
    y_train (pd.Series) - целевая переменная в обучении
    x_val (pd.DataFrame) - датафрейм для тестирования точности модели
    y_train (pd.Series) - целевая переменная в тестировании
    cat_names (pd.core.indexes.base.Index) - список категориальных признаков
    из тренировочного и тестового датафреймов для автоматической предобработки
    самой моделью
    model_name (str) - имя модели для сохранения
    verbose_step (int) - шаг вывода статуса модели

    return:
    Возвращается важность признаков модели
    """
    model.fit(
        x_train, y_train,                   # обучающая выборка
        cat_features=np.array(cat_names),   # категориальные признаки
        eval_set=(x_val, y_val),            # тестовая выборка
        verbose=verbose_step                # шаг вывода статуса модели
    )
    # сохранение модели
    model.save_model(f'{model_name}.json')
    # получение важности признаков
    feature_importance = model.get_feature_importance(prettified=True)
    return feature_importance

In [26]:
# получаем список категориальных признаков
cat_names = X.select_dtypes(include=['object']).columns
# обучение модели с получением важности признаков
feature_importance = train_catboost(
    catboost_model_start_cluster, X_train, y_train, X_val, y_val, cat_names,
    'catboost_model_start_cluster')

0:	learn: 1.9166969	test: 1.8946141	best: 1.8946141 (0)	total: 7.77s	remaining: 2h 12m 28s
100:	learn: 0.2481593	test: 0.2259391	best: 0.2259391 (100)	total: 12s	remaining: 1m 49s
200:	learn: 0.2268443	test: 0.2140390	best: 0.2140390 (200)	total: 16.1s	remaining: 1m 5s
300:	learn: 0.2194208	test: 0.2109518	best: 0.2109518 (300)	total: 19.9s	remaining: 47.8s
400:	learn: 0.2143160	test: 0.2090905	best: 0.2090905 (400)	total: 23.8s	remaining: 36.9s
500:	learn: 0.2104311	test: 0.2080493	best: 0.2080493 (500)	total: 27.5s	remaining: 28.8s
600:	learn: 0.2062180	test: 0.2068546	best: 0.2068546 (600)	total: 31.3s	remaining: 22.1s
700:	learn: 0.2025639	test: 0.2062209	best: 0.2062209 (700)	total: 35.2s	remaining: 16.2s
800:	learn: 0.1989897	test: 0.2054518	best: 0.2054488 (793)	total: 39.1s	remaining: 10.9s
900:	learn: 0.1957841	test: 0.2050265	best: 0.2050231 (899)	total: 43s	remaining: 5.87s
1000:	learn: 0.1926107	test: 0.2046149	best: 0.2046017 (997)	total: 46.9s	remaining: 1.08s
1023:	learn

In [27]:
# предсказываем разные классы из тестовой подвыборки
y_pred = catboost_model_start_cluster.predict(X_val)
# статистика точности моделей по разным метрикам
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

     {other}       0.93      0.92      0.92      2432
          {}       0.88      0.86      0.87      3847
      {α, β}       0.91      0.93      0.92       710
      {α, γ}       0.94      0.93      0.93      2274
      {α, δ}       0.86      0.86      0.86       575
   {α, ε, η}       0.92      0.89      0.90       125
   {α, ε, θ}       0.88      0.83      0.85        53
   {α, ε, ψ}       0.88      0.83      0.85        35
      {α, ε}       0.90      0.83      0.87       369
      {α, η}       0.96      0.97      0.97      3651
      {α, θ}       0.92      0.88      0.90       322
      {α, λ}       0.77      0.85      0.81        67
      {α, μ}       0.85      0.84      0.84        93
      {α, π}       0.00      0.00      0.00         1
      {α, ψ}       0.94      0.92      0.93       331
         {α}       0.96      0.97      0.97     25112
         {λ}       1.00      0.33      0.50         3

    accuracy              

#### Восстановление start_cluster для 6 месяца из тестовых данных

In [28]:
X_predict = predict_data.drop('start_cluster_month_3', axis=1)
predicted_clusters = catboost_model_start_cluster.predict(X_predict)

In [29]:
predicted_clusters_flat = np.ravel(predicted_clusters)
class_counts = pd.Series(predicted_clusters_flat).value_counts()
print(class_counts)

{α}          68414
{α, η}        8038
{}            6707
{other}       5743
{α, γ}        5094
{α, β}        1957
{α, δ}        1334
{α, ε}         787
{α, θ}         698
{α, ψ}         443
{α, μ}         270
{α, ε, η}      202
{α, λ}         148
{α, ε, θ}      113
{α, ε, ψ}       44
{λ}              8
Name: count, dtype: int64


In [30]:
predicted_index = 0

df_restore_start_cluster = df.copy()
for index, row in df_restore_start_cluster.iterrows():
    # Для клиентов test
    if row['id'] >= 200000:
        # Вставляем значение из серии в столбец 'start_cluster_month_3' текущей строки
        df_restore_start_cluster.at[index,
                                    'start_cluster_month_3'] = predicted_clusters[predicted_index][0]
        # Увеличиваем индекс текущей строки в серии
        predicted_index += 1

In [31]:
matching_rows = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000].loc[(df_restore_start_cluster['start_cluster_month_1'] == df_restore_start_cluster['start_cluster_month_2']) & (
    df_restore_start_cluster['start_cluster_month_2'] == df_restore_start_cluster['start_cluster_month_3'])]
matching_rows

,id,avg_a_oper_1m_month_1,avg_a_oper_1m_month_2,avg_a_oper_1m_month_3,avg_a_oper_3m_month_1,avg_a_oper_3m_month_2,avg_a_oper_3m_month_3,avg_b_oper_1m_month_1,avg_b_oper_1m_month_2,avg_b_oper_1m_month_3,...,sum_deb_h_oper_3m_month_3,sum_of_paym_1y_month_1,sum_of_paym_1y_month_2,sum_of_paym_1y_month_3,sum_of_paym_2m_month_1,sum_of_paym_2m_month_2,sum_of_paym_2m_month_3,sum_of_paym_6m_month_1,sum_of_paym_6m_month_2,sum_of_paym_6m_month_3
200000,200000,1.2619,-0.1648,6.2595,4.4611,4.6360,5.4649,-0.0693,-0.0693,-0.0693,...,-0.1528,0.6766,0.6884,0.6719,0.4168,0.4332,0.2240,0.3324,0.2843,0.2854
200001,200001,-0.4493,-0.4493,-0.4493,-0.9794,-0.9794,-0.9794,-0.0690,-0.0690,-0.0690,...,-0.1656,0.0035,0.0035,0.0035,0.0017,0.0017,0.0017,-0.0015,-0.0015,-0.0015
200002,200002,9.3303,43.0412,24.7229,16.9913,45.0169,49.5095,-0.0693,-0.0693,-0.0693,...,2.6149,0.3656,0.9705,1.2116,1.3040,3.8709,4.1425,0.5504,1.6208,1.9696
200003,200003,-0.4493,-0.4493,-0.4493,-0.9794,-0.9794,-0.9794,-0.0690,-0.0690,-0.0690,...,-0.1656,0.0035,0.0035,0.0035,0.0017,0.0017,0.0017,-0.0015,-0.0015,-0.0015
200005,200005,0.6094,-0.0917,-0.4102,-0.0926,0.0646,0.0782,-0.0693,-0.0693,-0.0693,...,0.2455,0.4250,0.1547,0.2579,0.0450,0.0677,0.3984,0.2243,0.1523,0.2306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299994,299994,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,-0.1656,-0.2656,-0.2704,-0.2707,-0.2562,-0.2631,-0.2680,-0.2625,-0.2793,-0.2823
299995,299995,-0.4493,-0.4493,-0.4493,-0.9794,-0.9794,-0.9794,-0.0690,-0.0690,-0.0690,...,-0.1656,-0.2961,-0.2961,-0.2961,-0.2740,-0.2740,-0.2740,-0.2946,-0.2946,-0.2946
299996,299996,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,-0.1558,-0.2894,-0.2898,-0.2875,-0.2735,-0.2720,-0.2599,-0.2845,-0.2838,-0.2842
299997,299997,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,...,0.0879,-0.1378,-0.1119,-0.0847,-0.0707,-0.0416,-0.0087,-0.0990,-0.0826,-0.0688


Также, как и в train, у нас не у всех клиентов совпадают start_cluster'ы. 

## Обучение модели <a class="anchor" id="fifth"></a>

Также CatBoost лучше всего себя показал и быстрее, что не мало важно. Мы пробовали LightGBM, XGBoost, многослойный перцептрон pytorch, ансамбль из моделей (CatBoost, LightGBM, XGBoost).

### Подготовка данных для обучения модели

In [32]:
# получаем обратно тренировочный и тестовый датасеты из общего
train_df = df_restore_start_cluster[df_restore_start_cluster['id']< 200000]
test_df = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000]
# получаем признаки для обучения и целевую переменную
X = train_df.drop(["id", "end_cluster_month_3"], axis=1)
y = train_df["end_cluster_month_3"]
# разбиваем на подвыборки обучения и тестирования 80/20
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [33]:
# архитектура модели
catboost_model_end_cluster = CatBoostClassifier(
    iterations=2025,             # кол-во итераций обучения
    depth=6,                     # рекомендованная глубина модели
    learning_rate=0.075,         # скорость обучения
    random_seed=47,              # сид для воспроизводимости результата
    loss_function='MultiClass',  # тип модели или функция ошибки
    task_type="GPU",             # обучение на видеокарте
    devices='0',
    early_stopping_rounds=20     # регуляризация ранней остановкой в случае
                                 # отстутсвия изменения функции ошибки 20
                                 # итераций подряд
    )

### Запуск обучения

In [34]:
cat_names = X_train.select_dtypes(include=['object']).columns

feature_importance = train_catboost(
    catboost_model_end_cluster, X_train, y_train, X_val, y_val, cat_names,
    'catboost_model_end_cluster')

0:	learn: 2.2483059	test: 2.2032299	best: 2.2032299 (0)	total: 49ms	remaining: 1m 39s
100:	learn: 0.8389839	test: 0.7675741	best: 0.7675741 (100)	total: 4.76s	remaining: 1m 30s
200:	learn: 0.8107581	test: 0.7524185	best: 0.7524185 (200)	total: 9.1s	remaining: 1m 22s
300:	learn: 0.7946256	test: 0.7469841	best: 0.7469841 (300)	total: 13.4s	remaining: 1m 16s
400:	learn: 0.7831790	test: 0.7443158	best: 0.7443158 (400)	total: 17.5s	remaining: 1m 10s
500:	learn: 0.7728747	test: 0.7426890	best: 0.7426742 (499)	total: 21.7s	remaining: 1m 5s
600:	learn: 0.7635568	test: 0.7415159	best: 0.7415159 (600)	total: 25.9s	remaining: 1m 1s
700:	learn: 0.7550114	test: 0.7403966	best: 0.7403966 (700)	total: 30s	remaining: 56.7s
800:	learn: 0.7470192	test: 0.7397764	best: 0.7397723 (799)	total: 34.3s	remaining: 52.4s
bestTest = 0.7393415527
bestIteration = 862
Shrink model to first 863 iterations.


### Важность признаков

In [35]:
feature_importance

,Feature Id,Importances
0,start_cluster_month_3,9.2473
1,okved_month_3,4.8910
2,segment_month_3,3.6744
3,index_city_code_month_3,3.5978
4,city_month_3,3.4531
...,...,...
329,cnt_days_cred_g_oper_1m_month_1,0.0000
330,cnt_days_cred_g_oper_1m_month_2,0.0000
331,cnt_days_cred_h_oper_1m_month_1,0.0000
332,sum_a_oper_3m_month_2,0.0000


### Прогнозирование классов на тестовой выборке

In [36]:
# предсказываем разные классы из тестовой подвыборки
y_pred = catboost_model_end_cluster.predict(X_val)
# статистика точности моделей по разным метрикам
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

     {other}       0.66      0.54      0.60      3381
          {}       0.72      0.69      0.70      7488
      {α, β}       0.57      0.31      0.40       772
      {α, γ}       0.65      0.64      0.64      2114
      {α, δ}       0.30      0.05      0.09       335
   {α, ε, η}       0.56      0.36      0.44       148
   {α, ε, θ}       0.50      0.12      0.19        76
   {α, ε, ψ}       0.33      0.03      0.06        32
      {α, ε}       0.45      0.25      0.32       343
      {α, η}       0.76      0.88      0.82      3050
      {α, θ}       0.59      0.44      0.50       385
      {α, λ}       0.37      0.09      0.15        75
      {α, μ}       0.69      0.32      0.44       129
      {α, π}       0.00      0.00      0.00         1
      {α, ψ}       0.51      0.59      0.55       262
         {α}       0.80      0.86      0.83     21405
         {λ}       1.00      0.25      0.40         4

    accuracy              

## Тестирование модели <a class="anchor" id="sixth"></a>

In [37]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [7]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [40]:
y_pred_proba = catboost_model_end_cluster.predict_proba(X_val)
weighted_roc_auc(y_val, y_pred_proba, catboost_model_end_cluster.classes_, weights_dict)

0.9174705953212728

Прогноз на тестовой выборке

In [10]:
sample_submission_df = pd.read_csv("sample_submission.csv") # поменять на свой
last_m_test_df = test_df
last_m_test_df = last_m_test_df.drop(["id" , 'end_cluster_month_3'], axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = catboost_model_end_cluster.predict_proba(pool2)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=catboost_model_end_cluster.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("final1.csv", index=False) # сохранение модели

In [42]:
sample_submission_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.0104,0.0144,0.0211,0.0229,0.0056,0.0003,0.0055,0.0003,0.0084,0.0038,0.0215,0.0011,0.0037,0.0000,0.0022,0.8787,0.0000
1,200001,0.0054,0.5187,0.0010,0.0016,0.0005,0.0002,0.0004,0.0000,0.0013,0.0065,0.0022,0.0002,0.0007,0.0000,0.0006,0.4604,0.0002
2,200002,0.5495,0.0051,0.0041,0.0745,0.0138,0.0049,0.0045,0.0214,0.0385,0.0107,0.0274,0.0155,0.0046,0.0000,0.0694,0.1561,0.0000
3,200003,0.0277,0.5884,0.0004,0.0014,0.0003,0.0006,0.0004,0.0000,0.0010,0.0144,0.0027,0.0000,0.0012,0.0000,0.0005,0.3608,0.0000
4,200004,0.0882,0.1086,0.0455,0.0228,0.0092,0.0039,0.0009,0.0001,0.0108,0.0660,0.0048,0.0008,0.0343,0.0000,0.0006,0.6035,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.0146,0.3165,0.0014,0.0049,0.0015,0.0000,0.0005,0.0000,0.0014,0.0014,0.0013,0.0004,0.0004,0.0000,0.0012,0.6543,0.0001
99996,299996,0.0185,0.0521,0.0085,0.0295,0.0071,0.0003,0.0005,0.0001,0.0075,0.0060,0.0081,0.0013,0.0024,0.0000,0.0019,0.8556,0.0005
99997,299997,0.0337,0.0368,0.0349,0.0545,0.0146,0.0001,0.0009,0.0005,0.0144,0.0035,0.0071,0.0010,0.0014,0.0000,0.0230,0.7737,0.0000
99998,299998,0.0931,0.0916,0.0243,0.0500,0.0115,0.0007,0.0024,0.0013,0.0238,0.0119,0.0096,0.0122,0.0019,0.0000,0.0033,0.6609,0.0014


---
## Выводы <a class="anchor" id="seventh"></a>

Наша задача заключалась в создании модели CLTV, способной предсказывать вероятности перехода клиентов в каждый из 17 продуктовых кластеров в течение 12 месяцев.

Для этого мы использовали данные, предоставленные Альфа-Банком:
- Тренировочный датасет `train_data.pqt`, содержащий информацию о 200 000 клиентах банка и их целевых переменных за три последовательных месяца (month_1, month_2, month_3).
- Тестовый датасет `test_data.pqt`, включающий записи о 100 000 клиентах за 3 последовательных месяца (month_4, month_5, month_6).
- Для каждого клиента указан продуктовый кластер, в который он, предположительно, будет принадлежать через год (`end_cluster`). Наша задача - предсказать вероятности перехода клиентов в эти продуктовые кластеры для последнего месяца (month_6).

В результате анализа данных мы обнаружили следующие закономерности:
- Клиент остаётся в том же кластере с 2 по 3 месяц: 93.3%
- Данные имели пропуски с определенной закономерностью

Мы создали новые признаки со средними значениями из комбинации других столбцов. Также мы объединили данные об одном клиенте в единую строку.


Качество модели оценивалось по метрике **ROC-AUC**.

Лучший результат по качеству и скорости показала модель градиентного бустинга **CatBoost**.  <br>
Полученный результат: ROC-AUC = **0.9039141107**, на всей выборке. <br>
Полученный результат: ROC-AUC = **0.896941145**, на публичной выборке (1 место).

